In [ ]:
%pip install "snowflake-connector-python[pandas]"
%pip install pandas
%pip install python-dotenv

In [ ]:
import os, logging
import pandas as pd
import snowflake.connector

In [ ]:
%load_ext dotenv
%dotenv

# RELIES ON .env FOR RETIIEVING CREDENTIALS


SNOW_PASS = os.getenv('sdl_pwd')
SNOW_USER = os.getenv('sdl_user')
WAREHOUSE = os.getenv('sdl_warehouse')
ACCOUNT = os.getenv('acct_id')

In [ ]:
conn = snowflake.connector.connect(
    user=SNOW_USER,
    password=SNOW_PASS,
    account=ACCOUNT,
    authenticator="snowflake",
    database='BUS_BATCAVE_SECURITY_DB',
    warehouse='BATCAVESECURITY_WH',
    role="SVC_BATCAVESECURITY_ROLE",
    schema="PUBLIC",
    autocommit=True,
    login_timeout=60,
    network_timeout=30,
    socket_timeout=15
)


In [ ]:
cur = conn.cursor()

In [ ]:

cur.execute("select TITLE, AWS_ACCOUNT_NAME, FINDINGPROVIDERFIELDS_SEVERITY_LABEL from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID WHERE RECORDSTATE='ACTIVE' and WORKFLOW_STATUS='NEW' and PRODUCTNAME not in ('default', 'Inspector','GuardDuty') and CREATEDAT >= CURRENT_TIMESTAMP() - INTERVAL '24 hours'")

df = cur.fetch_pandas_all()
for row in df.iterrows():
    print(row[1][0])

In [ ]:
# conn.cursor().execute("SHOW WAREHOUSES")
# row=conn.cursor().fetch()
# print(row)

cur.execute("select DISTINCT TITLE from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID WHERE RECORDSTATE='ACTIVE' and PRODUCTNAME not in ('Default', 'Inspector')")

df = cur.fetch_pandas_all()
df

In [ ]:
cur.execute("select distinct(AWS_ACCOUNT_ID), AWS_ACCOUNT_NAME from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID")
df = cur.fetch_pandas_all()
df

In [ ]:
cur.execute("select count(WORKFLOW_STATUS), WORKFLOW_STATUS from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID GROUP BY WORKFLOW_STATUS")
df = cur.fetch_pandas_all()
df

In [ ]:
cur.execute("select FINDINGPROVIDERFIELDS_TYPES from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID WHERE RECORDSTATE='ACTIVE'")

df = cur.fetch_pandas_all()
df

In [ ]:
cur.execute("select * from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID where AWS_ACCOUNT_NAME = 'batcave-prod' and PRODUCTNAME ='Security Hub' and AWS_ACCOUNT_ID='863306670509' and WORKFLOW_STATUS='NEW' and RECORDSTATE='ACTIVE'")

df = cur.fetch_pandas_all()
df
#df.to_csv('sechub_batcave_prod.csv')

In [ ]:
cur.execute("select * from BUS_BATCAVE_SECURITY_DB.PUBLIC.SEC_VW_COMMERCIAL_SECHUB_FINDINGS_BY_BATCAVE_ACCOUNT_ID WHERE RECORDSTATE='ACTIVE' and WORKFLOW_STATUS='NEW' and PRODUCTNAME not in ('Default', 'Inspector', 'GuardDuty')")
df = cur.fetch_pandas_all()
df.to_csv('sechub_all_open.csv')

In [ ]:
cur.execute("select * from SEC_VW_VULN_AWS_BATCAVE WHERE SEVERITY_NAME in  ('Critical') and ACCOUNTID='863306670509' and ARRAY_SIZE(CVE)>1")
df = cur.fetch_pandas_all()
df.to_csv("multiple_cves.csv")

In [ ]:
cur.execute("select INSTANCEID, SEVERITY_NAME, count(INSTANCEID) from SEC_VW_VULN_AWS_BATCAVE where ACCOUNTID='863306670509' and report_date>current_date-7 group by INSTANCEID, SEVERITY_NAME ORDER BY INSTANCEID")
df = cur.fetch_pandas_all()
df